# Install Required Software or Run a Docker Image

<br>
    
This notebook installs the Pixi software environment required to run the other notebooks in the NISAR GCOV Cookbook.

<hr>

## Overview

1. [Prerequisites](env-prereqs)
1. [(Option 1) Install the environment with the Pixi pacakage manager](env-pixi)
1. [(Option 2) Run a Docker Image with the software environment already installed](env-docker)
1. [Summary](env-summary)
1. [Resources and references](env-resources)

<hr>

(env-prereqs)=
## 1. Prerequisites

| Prerequisite | Importance | Notes |
| --- | --- | --- |
| [Option 1: The Pixi package manager must be installed on the system running Jupyter Lab](https://pixi.sh/latest/installation/) | Necessary | Pixi may not be supported on all Jupyter Hubs|
| [Option 2: Docker must be installed and running on the system running Jupyter Lab](https://www.docker.com/get-started/) | Necessary | Run the Docker image locally or in a Jupyter Hub |
| [UW Scientific Software Engineering Center's Pixi guide](https://rse-guidelines.readthedocs.io/en/latest/fundamentals/computing-development-environments/pixi/) | Helpful | This is a great resource to get quickly get started with Pixi|

- **Rough Notebook Time Estimate**: 3 minutes

<hr>

(env-pixi)=
## 2. (Option 1) Install the environment with the Pixi pacakage manager

### 2a. Create a `work_dir` context manager

This is useful when you want to run a command in a working directory and then automatically change back to your original directory 

In [ ]:
import os
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def work_dir(new_dir):
    old_dir = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(old_dir)

### 2b. Install the `isce3` environment with Pixi

In [ ]:
with work_dir(Path.cwd().parent):
    !pixi install -e isce3

### 2c. Register the `isce3` environment's Python kernel with `ipykernel`

In [ ]:
env_name = "isce3"
display_name = f'"{env_name} (Python)"'

!pixi run -e isce3 python -m ipykernel install \
  --user \
  --name $env_name \
  --display-name $display_name

### 2d. Ensure notebook shell commands run in the `isce3` environment

This ensures that shell commands executed from inside a notebook with `!` run in the notebook kernel’s environment. This works by launching the entire Jupyter kernel process inside the Pixi environment, so the kernel’s PATH, environment variables, and Python executable all come from that Pixi environment, not from the parent JupyterLab environment.

In [ ]:
from jupyter_client.kernelspec import KernelSpecManager
import json

ksm = KernelSpecManager()
spec = ksm.get_kernel_spec(env_name)
kernel_dir = Path(spec.resource_dir)
kernel_json = kernel_dir / "kernel.json"

data = json.loads(kernel_json.read_text())
orig_argv = data.get("argv", [])

new_argv = [
    "pixi",
    "run",
    "--manifest-path",
    str(Path.cwd().parent),
    "-e",
    env_name,
] + orig_argv

data["argv"] = new_argv
kernel_json.write_text(json.dumps(data, indent=2))
print(f"Updated kernel.json at {kernel_json} with Pixi wrapper.")
print("argv:", data["argv"])

### 2e. (Optional) Delete the environment and remove its `kernelspec`

In [ ]:
with work_dir(Path.cwd().parent):
    !pixi clean
    !jupyter kernelspec remove isce3 -y

<hr>

(env-docker)=
## 3. (Option 2) Run a Docker Image with the software environment already installed

Coming soon


(env-summary)=
## 4. Summary
Now that you have installed the software environment or have the Docker image running, [make sure you have access to the data](https://github.com/ASFOpenSARlab/NISAR_GCOV_Cookbook/blob/main/notebooks/set_up_Earthdata_Login.md). You will then be ready to run the remaining notebooks in the NISAR GCOV Cookbook.
<hr>

(env-resources)=
## 5. Resources and references

### References
- [{abbr}`UW SSEC (Univertsity of Washington Scientific Software Engineering Center)`](https://escience.washington.edu/software-engineering/ssec/)

**Author:** Alex Lewandowski